In [7]:
import gradio as gr
import requests

def get_token():
    endpoint = "https://eastus.api.cognitive.microsoft.com/sts/v1.0/issueToken"
    api_key = "8e5931bade634c4e859a8e7544f87ff7"

    headers = {
        "Ocp-Apim-Subscription-Key": api_key,
    }
    
    response = requests.post(endpoint,
                             headers=headers)
    
    if response.status_code == 200:
        access_token = response.text
        return access_token
    else:
        return ''

def request_tts(text):
    endpoint = "https://eastus.tts.speech.microsoft.com/cognitiveservices/v1"
    access_token = get_token()
   
    headers = {
        "Content-Type": "application/ssml+xml",
        "User-Agent": "testForEducation",
        "X-Microsoft-OutputFormat": "riff-24khz-16bit-mono-pcm",
        "Authorization": f"Bearer {access_token}"
    }

    data = f"""
    <speak version='1.0' xml:lang='ko-KR'><voice xml:lang='ko-KR' xml:gender='Female' name='ko-KR-JiMinNeural'>
        {text}
    </voice></speak>
    """

    response = requests.post(endpoint,
                             headers=headers,
                             data=data)
    
    if response.status_code == 200:
        file_name = 'response_audio.wav'
        with open(file_name, "wb") as audio_file:
            audio_file.write(response.content)
        return file_name
    else:
        return None

def click_tts(text):

    file_name = request_tts(text)

    if file_name:
        return file_name
    else:
        return None

with gr.Blocks() as demo:
    with gr.Column():
        gr.Markdown('<h3>Text To Speech</h3>')
        input_textbox = gr.Textbox(label="입력 텍스트", placeholder="텍스트를 입력하세요")
        send_button = gr.Button("전송")
        output_audio = gr.Audio(label="출력 오디오", type="filepath", interactive=False)

        send_button.click(fn=click_tts, inputs=[input_textbox], outputs=[output_audio])


demo.launch()

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
